# 10. Models with inputs

In this notebook we evaluate how exogenous inputs (engineered features) improve
the performance of the developed forecasting models.

We will compare:
- a purely autoregressive statistical model **SARIMA(1,1,1)(1,1,1,24)**, and
- a machine learning model **XGBoost** that uses time and weather features.

The evaluation is done on the **forecast.csv** period using **MAE** and **nRMSE**.

## Setup and imports

In [1]:
from functions import *

## Load and preprocess the datasets

We will reuse the same cleaning pipeline as in Tasks 8–9.

In [2]:
# train dataset
train_raw = load_data()
train_raw = train_raw.sort_index()
train_raw = train_raw.asfreq("h")
train_raw = train_raw.interpolate(method="time", limit_direction="both")
train_raw = train_raw.reset_index()
train_raw = train_raw.round(5)

# forecast dataset
forecast_raw = load_forecast_data()
forecast_raw = forecast_raw.sort_index()
forecast_raw = forecast_raw.asfreq("h")
forecast_raw = forecast_raw.interpolate(method="time", limit_direction="both")
forecast_raw = forecast_raw.reset_index()
forecast_raw = forecast_raw.round(5)


show_table_info(train_raw, "Train data")
show_table_info(forecast_raw, "Forecast data")


TRAIN DATA SUMMARY
Shape: 8,760 rows × 18 columns
Time span: 2013-07-01 00:00:00+00:00 -> 2014-06-30 23:00:00+00:00

                         Column    Type   NA %
                        pv_mod1 float64  0.00%
                        pv_mod2 float64  0.00%
                        pv_mod3 float64  0.00%
                         demand float64  0.00%
                             pv float64  0.00%
                          price float64  0.00%
                    temperature float64  0.00%
                 pressure (hPa) float64  0.00%
                cloud_cover (%) float64  0.00%
            cloud_cover_low (%) float64  0.00%
            cloud_cover_mid (%) float64  0.00%
           cloud_cover_high (%) float64  0.00%
          wind_speed_10m (km/h) float64  0.00%
     shortwave_radiation (W/m²) float64  0.00%
        direct_radiation (W/m²) float64  0.00%
       diffuse_radiation (W/m²) float64  0.00%
direct_normal_irradiance (W/m²) float64  0.00%


FORECAST DATA SUMMARY
Shape: 168 r

,Column,Type,NA %
0,demand,float64,0.00%
1,pv,float64,0.00%
2,price,float64,0.00%
3,temperature,float64,0.00%
4,pressure (hPa),float64,0.00%
5,cloud_cover (%),int64,0.00%
6,cloud_cover_low (%),int64,0.00%
7,cloud_cover_mid (%),int64,0.00%
8,cloud_cover_high (%),int64,0.00%
9,wind_speed_10m (km/h),float64,0.00%


## Add engineered features
Here we add the same time and weather based features that were used in the
XGBoost model in Notebook 8.

In [3]:
train_fe = add_time_weather_features(train_raw)
forecast_fe = add_time_weather_features(forecast_raw)

train_fe = train_fe.round(5)
forecast_fe = forecast_fe.round(5)

show_table_info(train_fe, "Train with features")
show_table_info(forecast_fe, "Forecast with features")



TRAIN WITH FEATURES SUMMARY
Shape: 8,760 rows × 25 columns
Time span: 2013-07-01 00:00:00+00:00 -> 2014-06-30 23:00:00+00:00

                         Column    Type   NA %
                        pv_mod1 float64  0.00%
                        pv_mod2 float64  0.00%
                        pv_mod3 float64  0.00%
                         demand float64  0.00%
                             pv float64  0.00%
                          price float64  0.00%
                    temperature float64  0.00%
                 pressure (hPa) float64  0.00%
                cloud_cover (%) float64  0.00%
            cloud_cover_low (%) float64  0.00%
            cloud_cover_mid (%) float64  0.00%
           cloud_cover_high (%) float64  0.00%
          wind_speed_10m (km/h) float64  0.00%
     shortwave_radiation (W/m²) float64  0.00%
        direct_radiation (W/m²) float64  0.00%
       diffuse_radiation (W/m²) float64  0.00%
direct_normal_irradiance (W/m²) float64  0.00%
                           

,Column,Type,NA %
0,demand,float64,0.00%
1,pv,float64,0.00%
2,price,float64,0.00%
3,temperature,float64,0.00%
4,pressure (hPa),float64,0.00%
5,cloud_cover (%),int64,0.00%
6,cloud_cover_low (%),int64,0.00%
7,cloud_cover_mid (%),int64,0.00%
8,cloud_cover_high (%),int64,0.00%
9,wind_speed_10m (km/h),float64,0.00%


## Select feature set for the ML model

In [4]:
FEATURES = [
    "timestamp", "demand", "hour_sin", "hour_cos", "is_weekend",
    "cooling_degree", "heating_degree", "temperature",
    "pressure (hPa)", "cloud_cover (%)", "wind_speed_10m (km/h)",
    "shortwave_radiation (W/m²)", "direct_radiation (W/m²)",
    "diffuse_radiation (W/m²)", "direct_normal_irradiance (W/m²)",
    "price",
]

train_fe = train_fe[FEATURES]
forecast_fe = forecast_fe[FEATURES]

train_fe.head()

,timestamp,demand,hour_sin,hour_cos,is_weekend,cooling_degree,heating_degree,temperature,pressure (hPa),cloud_cover (%),wind_speed_10m (km/h),shortwave_radiation (W/m²),direct_radiation (W/m²),diffuse_radiation (W/m²),direct_normal_irradiance (W/m²),price
0,2013-07-01 00:00:00+00:00,0.27,0.00000,1.00000,0,0.0,4.5,13.5,1011.3,4.0,10.5,0.00000,0.0,0.0,0.0,0.01605
1,2013-07-01 01:00:00+00:00,0.23,0.25882,0.96593,0,0.0,4.8,13.2,1010.8,27.0,11.9,0.00000,0.0,0.0,0.0,0.00095
2,2013-07-01 02:00:00+00:00,0.26,0.50000,0.86603,0,0.0,4.9,13.1,1010.3,33.0,11.6,0.00000,0.0,0.0,0.0,0.00060
3,2013-07-01 03:00:00+00:00,0.28,0.70711,0.70711,0,0.0,5.0,13.0,1010.3,28.0,11.2,51.45455,2.0,7.0,30.1,0.00046
4,2013-07-01 04:00:00+00:00,0.29,0.86603,0.50000,0,0.0,4.2,13.8,1010.2,16.0,11.7,102.90909,30.0,31.0,252.0,0.00046


## Autoregressive statistical model – SARIMA

As an autoregressive baseline we use the SARIMA model that was developed earlier:

- SARIMA(1,1,1)(1,1,1,24)

The model is trained on the full training period and then used to forecast
the whole `forecast.csv` horizon (7 days of hourly demand).


In [5]:
# Define SARIMA configuration (from Task 7)
SARIMA_ORDER = (1, 1, 1)
SARIMA_SEASONAL = (1, 1, 1, 24)

# Train SARIMA on the full training demand series
sarima_series = train_fe.set_index("timestamp")["demand"]

sarima_model = fit_arima(
    series=sarima_series,
    order=SARIMA_ORDER,
    seasonal_order=SARIMA_SEASONAL,
)

# Forecast over the entire forecast.csv horizon
sarima_fc = forecast_arima(
    model=sarima_model,
    horizon=len(forecast_fe),
    index=forecast_fe["timestamp"],
)

# Evaluate SARIMA on forecast period
sarima_metrics = evaluate_forecast(
    y_true=forecast_fe["demand"].values,
    y_pred=sarima_fc.values,
)
sarima_metrics


C:\Users\danyi\Desktop\TALTECH magister\Andmeteadus Energeetikas\hems\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


{'MAE': 0.21330147217442247,
 'RMSE': np.float64(0.2971672725032863),
 'nRMSE': np.float64(0.16063095810979766),
 'MAPE': 0.5158394909542741}

## ML model with inputs – XGBoost

Next we train an XGBoost regression model that uses:

- past demand, and
- exogenous time and weather features

The model is trained on the full training dataset and evaluated on the
entire forecast horizon.


In [6]:
X_train = train_fe.drop(columns=["demand", "timestamp"])
y_train = train_fe["demand"]

X_test = forecast_fe.drop(columns=["demand", "timestamp"])
y_test = forecast_fe["demand"]

xgb_model, xgb_history = train_xgboost(X_train, y_train)

xgb_pred = xgb_model.predict(X_test)

xgb_metrics = evaluate_forecast(y_test.values, xgb_pred)
xgb_metrics

{'MAE': 0.17977529798590003,
 'RMSE': np.float64(0.273294890947021),
 'nRMSE': np.float64(0.14772696807939095),
 'MAPE': 0.40793506132047286}

## Compare performance: SARIMA vs XGBoost

We now compare the statistical autoregressive model (SARIMA) and the ML model
with exogenous inputs (XGBoost) using the MAE and nRMSE metrics.


In [7]:
comparison = pd.DataFrame([
    {"Model": "SARIMA(1,1,1)(1,1,1,24)", **sarima_metrics},
    {"Model": "XGBoost (with features)", **xgb_metrics},
])

comparison.round(4)

,Model,MAE,RMSE,nRMSE,MAPE
0,"SARIMA(1,1,1)(1,1,1,24)",0.2133,0.2972,0.1606,0.5158
1,XGBoost (with features),0.1798,0.2733,0.1477,0.4079


## Improvement of the model with exogenous features

Below we quantify how much the XGBoost model (with time- and weather-based inputs) improves over the purely autoregressive SARIMA model.
Both absolute and percentage improvements are reported for MAE and nRMSE.
Positive values indicate that the feature-based model performs better.

In [8]:
# Absolute improvements
improve_mae = sarima_metrics["MAE"] - xgb_metrics["MAE"]
improve_nrmse = sarima_metrics["nRMSE"] - xgb_metrics["nRMSE"]

# Percentage improvements
pct_mae = improve_mae / sarima_metrics["MAE"] * 100
pct_nrmse = improve_nrmse / sarima_metrics["nRMSE"] * 100

improvement = pd.DataFrame([
    {
        "Metric": "MAE",
        "Improvement": improve_mae,
        "Improvement %": pct_mae,
    },
    {
        "Metric": "nRMSE",
        "Improvement": improve_nrmse,
        "Improvement %": pct_nrmse,
    },
])

improvement.round(4)


,Metric,Improvement,Improvement %
0,MAE,0.0335,15.7177
1,nRMSE,0.0129,8.0333


## Feature importance
We extract and rank XGBoost feature importances to see which engineered features have the largest impact on the forecast.

In [9]:
# Extract feature importances
importances = xgb_model.feature_importances_
feature_names = X_train.columns

imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

imp_df


,feature,importance
1,hour_cos,0.134723
0,hour_sin,0.090170
5,temperature,0.080042
13,price,0.075065
11,diffuse_radiation (W/m²),0.073491
4,heating_degree,0.072071
8,wind_speed_10m (km/h),0.064834
12,direct_normal_irradiance (W/m²),0.062400
9,shortwave_radiation (W/m²),0.062334
6,pressure (hPa),0.062116


In [10]:
fig_imp = go.Figure()

fig_imp.add_trace(go.Bar(
    x=imp_df["feature"],
    y=imp_df["importance"],
    marker_color=ENERGY_COLORS["solar"]
))

fig_imp.update_layout(
    title="XGBoost Feature Importance",
    xaxis_title="Feature",
    yaxis_title="Importance",
    xaxis_tickangle=45,
    **PLOT_STYLE
)

save_fig_plotly(fig_imp, "ex10_fig1_feature_importance.svg", width=1100, height=500)
fig_imp.show()


## Conclusion
In this task we compared a pure autoregressive model (SARIMA) with a model that uses additional time- and weather-based features (XGBoost). The results show that adding exogenous inputs clearly improves forecast accuracy: MAE drops by about 16% and nRMSE by about 8%.

The feature importance analysis explains this improvement — variables like hour-of-day (sin/cos), temperature, and price play a major role in shaping electricity demand, and XGBoost is able to use this information effectively, unlike SARIMA.